In [ ]:
import pandas as pd
import os
import nltk
import datetime as dt
import numpy as np
from datetime import timedelta
from utils import preprocess_text

In [ ]:
df_ECT_500 = pd.read_csv("df_ECT_500.csv")

In [ ]:
df_ECT_500

In [ ]:
def processTranscript(input_data):    
    core_transcript=''
    if type(input_data) == str:
        transcript=input_data.split('\n')
        #for transcript in input_data:
        record= False  
        # print(type(input_data))
        for line in transcript:
            #print (f'{line_num}: \n {line}')
            if line.strip() == 'Operator':  # Or whatever test is needed
                #break
                record=True
            if line.strip() == 'Question-and-Answer Session':
                #record=False
                break
            if record: 
                core_transcript = core_transcript + ' \n' + line
        return core_transcript
    else:
        return "error"

In [ ]:
print(processTranscript(df_ECT_500.loc[0]['body']))

In [ ]:
from utils import preprocess_text

In [ ]:
processed = preprocess_text(processTranscript(df_ECT_500.loc[0]['body']))
print(processed)

In [ ]:
df_ECT_500['prepared'] = df_ECT_500.apply(lambda x: processTranscript(x['body']), axis=1)

In [ ]:
df_ECT_500 = df_ECT_500.drop(index=2953)

In [ ]:
df_ECT_500['prepared_str'] = df_ECT_500['prepared'].apply(lambda x : ' '.join(preprocess_text(x, min_word_length=4)))

In [ ]:
# Creating a dataframe with 80% 
# values of original dataframe 
train_df = df_ECT_500.sample(frac = 0.80) 
  
# Creating dataframe with  
# rest of the 20% values 
test_df = df_ECT_500.drop(train_df.index) 
  
train_df.shape

In [ ]:
# Vectorization methods
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
#vectorize training and test dataframe, passing bigrams and trigrams and using L1 

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3), norm='l1')
tfidf_vectorizer.fit(train_df['prepared_str'])
tfidf_train_vecs = tfidf_vectorizer.transform(train_df['prepared_str'])
tfidf_test_vecs = tfidf_vectorizer.transform(test_df['prepared_str'])

In [ ]:
from sklearn.linear_model import LogisticRegression
tfidf_logReg_1 = LogisticRegression(multi_class="auto", max_iter=100000)
tfidf_logReg_1.fit(tfidf_train_vecs, train_df['rating_1'])

# Calculate the percentage of accurate predictions
accuracy_tfidf_logReg_1 = tfidf_logReg_1.score(tfidf_test_vecs, test_df['rating_1'])
print(f"LogReg TF-IDF accuracy: {accuracy_tfidf_logReg_1}")

In [43]:
model = LogisticRegression(multi_class="auto")

In [49]:
param_grid = {'penalty': ['l1', 'l2'],
              'solver': ['lbfgs', 'newton-cg'],
              'C': [0.001, 0.01, 0.1, 1, 10, 100]}

grid = GridSearchCV(model, param_grid, verbose=3, scoring='f1', cv = 5)
X_train = tfidf_train_vecs
y_train = train_df['rating_1']

In [55]:
LR = LogisticRegression(multi_class="auto")
LRparam_grid = {
    'C': [10, 100, 1000, 10000],
    'penalty': ['l1'],
    'max_iter': [100],
    'solver': ['liblinear']
}
LR_search = GridSearchCV(LR, param_grid=LRparam_grid, refit = True, verbose = 3, cv=5)

# fitting the model for grid search 
LR_search.fit(X_train , y_train)
LR_search.best_params_
# summarize
print('Mean Accuracy: %.3f' % LR_search.best_score_)
print('Config: %s' % LR_search.best_params_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] C=10, max_iter=100, penalty=l1, solver=liblinear ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  C=10, max_iter=100, penalty=l1, solver=liblinear, score=0.739, total=  14.9s
[CV] C=10, max_iter=100, penalty=l1, solver=liblinear ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   14.8s remaining:    0.0s


[CV]  C=10, max_iter=100, penalty=l1, solver=liblinear, score=0.738, total=  14.4s
[CV] C=10, max_iter=100, penalty=l1, solver=liblinear ................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   29.2s remaining:    0.0s


[CV]  C=10, max_iter=100, penalty=l1, solver=liblinear, score=0.738, total=  14.7s
[CV] C=10, max_iter=100, penalty=l1, solver=liblinear ................
[CV]  C=10, max_iter=100, penalty=l1, solver=liblinear, score=0.739, total=  12.7s
[CV] C=10, max_iter=100, penalty=l1, solver=liblinear ................
[CV]  C=10, max_iter=100, penalty=l1, solver=liblinear, score=0.739, total=  14.4s
[CV] C=100, max_iter=100, penalty=l1, solver=liblinear ...............
[CV]  C=100, max_iter=100, penalty=l1, solver=liblinear, score=0.745, total=  21.3s
[CV] C=100, max_iter=100, penalty=l1, solver=liblinear ...............
[CV]  C=100, max_iter=100, penalty=l1, solver=liblinear, score=0.736, total=  22.7s
[CV] C=100, max_iter=100, penalty=l1, solver=liblinear ...............
[CV]  C=100, max_iter=100, penalty=l1, solver=liblinear, score=0.743, total=  26.4s
[CV] C=100, max_iter=100, penalty=l1, solver=liblinear ...............
[CV]  C=100, max_iter=100, penalty=l1, solver=liblinear, score=0.738, tot

[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed: 27.9min finished


Mean Accuracy: 0.740
Config: {'C': 100, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}


In [53]:
config = {'C': 100, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}

In [ ]:
vectors = tfidf_vectorizer.transform(df_ECT_500['prepared_str']) 

In [93]:
df_ECT_500

,title,url,date,Symbol,GICS Sector,GICS Sub-Industry,date_plus_28,date_plus_7,date_plus_1,Price,...,rating_28,rating_7,rating_1,body,body_tokens,body_str,prepared,prepared_str,tfidf_vecs,return_1_predict
0,3M Company (MMM) CEO Mike Roman on Q3 2020 Res...,/article/4381810-3m-company-mmm-ceo-mike-roman...,2020-10-27,MMM,Industrials,Industrial Conglomerates,2020-11-24,2020-11-03,2020-10-28,161.029999,...,buy,hold,hold,3M Company (NYSE:MMM) Q3 2020 Earnings Confere...,"['company', 'nyse', 'mmm', 'earn', 'conference...",company nyse earn conference call october comp...,"\nOperator \nLadies and gentlemen, thank you ...",operator lady gentleman thank stand welcome th...,"(0, 5791088)\t0.0003081618246638321\n (0, 5...","[hold, hold, hold, hold, hold, hold, hold, hol..."
1,"3M (MMM) CEO, Michael Roman on Q2 2020 Results...",/article/4361384-3m-mmm-ceo-michael-roman-on-q...,2020-07-28,MMM,Industrials,Industrial Conglomerates,2020-08-25,2020-08-04,2020-07-29,155.330002,...,buy,hold,hold,3M Company (NYSE:MMM) Q2 2020 Earnings Confere...,"['company', 'nyse', 'mmm', 'earn', 'conference...",company nyse earn conference call july company...,"\nOperator \nLadies and gentlemen, thank you ...",operator lady gentleman thank stand welcome se...,"(0, 5791266)\t0.0003405842794777901\n (0, 5...","[hold, hold, hold, hold, hold, hold, hold, hol..."
2,3M Company (MMM) CEO Mike Roman on Q1 2020 Res...,/article/4340658-3m-company-mmm-ceo-mike-roman...,2020-04-28,MMM,Industrials,Industrial Conglomerates,2020-05-26,2020-05-05,2020-04-29,157.610001,...,sell,sell,hold,3M Company (NYSE:MMM) Q1 2020 Earnings Confere...,"['company', 'nyse', 'mmm', 'earn', 'conference...",company nyse earn conference call april compan...,"\nOperator \nLadies and gentlemen, thank you ...",operator lady gentleman thank stand welcome fi...,"(0, 5791292)\t0.00045337130240940627\n (0, ...","[hold, hold, hold, hold, hold, hold, hold, hol..."
3,3M's (MMM) CEO Mike Roman on Q4 2019 Results -...,/article/4319588-3ms-mmm-ceo-mike-roman-on-q4-...,2020-01-28,MMM,Industrials,Industrial Conglomerates,2020-02-25,2020-02-04,2020-01-29,165.580002,...,sell,sell,hold,3M Company (NYSE:MMM) Q4 2019 Earnings Confere...,"['company', 'nyse', 'mmm', 'earn', 'conference...",company nyse earn conference call january comp...,"\nOperator \nLadies and gentlemen, thank you ...",operator lady gentleman thank stand welcome fo...,"(0, 5782557)\t0.00024771938828357297\n (0, ...","[hold, hold, hold, hold, hold, hold, hold, hol..."
4,3M Company (MMM) CEO Mike Roman on Q3 2019 Res...,/article/4298957-3m-company-mmm-ceo-mike-roman...,2019-10-24,MMM,Industrials,Industrial Conglomerates,2019-11-21,2019-10-31,2019-10-25,161.889999,...,hold,hold,hold,3M Company (NYSE:MMM) Q3 2019 Earnings Confere...,"['company', 'nyse', 'mmm', 'earn', 'conference...",company nyse earn conference call october comp...,"\nOperator \nLadies and gentlemen, thank you ...",operator lady gentleman thank stand welcome th...,"(0, 5792570)\t0.0003934249214704603\n (0, 5...","[hold, hold, hold, hold, hold, hold, hold, hol..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5268,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q1 ...",/article/4259182-zoetis-inc-zts-ceo-juan-ramon...,2019-05-02,ZTS,Health Care,Pharmaceuticals,2019-05-30,2019-05-09,2019-05-03,103.150002,...,hold,hold,hold,Zoetis Inc. (NYSE:ZTS) Q1 2019 Earnings Confer...,"['zoetis', 'inc.', 'nyse', 'zts', 'earn', 'con...",zoetis inc. nyse earn conference call company ...,\nOperator \nWelcome to the First Quarter 201...,operator welcome first quarter financial resul...,"(0, 5795569)\t0.00024693121016140915\n (0, ...","[hold, hold, hold, hold, hold, hold, hold, hol..."
5269,"Zoetis, Inc. (ZTS) CEO Juan Ramón Alaix on Q4 ...",/article/4241305-zoetis-inc-zts-ceo-juan-ramon...,2019-02-14,ZTS,Health Care,Pharmaceuticals,2019-03-14,2019-02-21,2019-02-15,93.260002,...,buy,hold,hold,"Zoetis, Inc. (NYSE:ZTS) Q4 2018 Earnings Confe...","['zoetis', 'inc.', 'nyse

In [91]:
df_ECT_500['tfidf_vecs'][0]

<1x5796539 sparse matrix of type '<class 'numpy.float64'>'
	with 3889 stored elements in Compressed Sparse Row format>

In [96]:
LR_search.predict(df_ECT_500['tfidf_vecs'][0])[0]

'hold'

In [86]:
len(list(df_ECT_500.iloc[0]['return_1_predict']))

5272

In [99]:
def return_1_predict(x):
    try:
        prediction = LR_search.predict(x)
        return prediction[0]
    except:
        return np.nan

In [102]:
df_ECT_500['predict_return_1_logReg'] = df_ECT_500['tfidf_vecs'].apply(lambda x: return_1_predict(x))
df_ECT_500['predict_return_1_logReg']



0       hold
1       hold
2       hold
3       hold
4       hold
        ... 
5268    hold
5269    hold
5270    hold
5271    hold
5272     NaN
Name: predict_return_1_logReg, Length: 5272, dtype: object

In [104]:
df_ECT_500 = df_ECT_500.drop(columns=['return_1_predict'])

In [112]:
df_ECT_500[df_ECT_500['rating_1'] == df_ECT_500['predict_return_1_logReg']].shape

(3930, 26)

In [113]:
df_ECT_500[(df_ECT_500['rating_1'] == 'sell') & (df_ECT_500['predict_return_1_logReg'] == 'sell')].shape

(35, 26)

In [114]:
df_ECT_500[(df_ECT_500['rating_1'] == 'buy') & (df_ECT_500['predict_return_1_logReg'] == 'buy')].shape

(47, 26)

In [120]:
df_ECT_500[(df_ECT_500['rating_1'] == 'sell') & (df_ECT_500['predict_return_1_logReg'] == 'buy')].shape

(26, 26)

In [119]:
df_ECT_500[(df_ECT_500['rating_1'] == 'buy') & (df_ECT_500['predict_return_1_logReg'] == 'sell')].shape

(21, 26)

In [115]:
df_ECT_500[df_ECT_500['rating_1'] != df_ECT_500['predict_return_1_logReg']].shape

(1342, 26)

In [117]:
df_ECT_500_predict = df_ECT_500.drop(columns=['tfidf_vecs', 'body', 'body_tokens', 'prepared', 'prepared_str'])

In [118]:
df_ECT_500_predict = df_ECT_500_predict.drop(columns=['body_str'])

,title,url,date,Symbol,GICS Sector,GICS Sub-Industry,date_plus_28,date_plus_7,date_plus_1,Price,...,price_plus_7,price_plus_1,return_28,return_7,return_1,rating_28,rating_7,rating_1,body_str,predict_return_1_logReg
0,3M Company (MMM) CEO Mike Roman on Q3 2020 Res...,/article/4381810-3m-company-mmm-ceo-mike-roman...,2020-10-27,MMM,Industrials,Industrial Conglomerates,2020-11-24,2020-11-03,2020-10-28,161.029999,...,165.509995,158.529999,0.099671,0.027821,-0.015525,buy,hold,hold,company nyse earn conference call october comp...,hold
1,"3M (MMM) CEO, Michael Roman on Q2 2020 Results...",/article/4361384-3m-mmm-ceo-michael-roman-on-q...,2020-07-28,MMM,Industrials,Industrial Conglomerates,2020-08-25,2020-08-04,2020-07-29,155.330002,...,151.210007,156.250000,0.050023,-0.026524,0.005923,buy,hold,hold,company nyse earn conference call july company...,hold
2,3M Company (MMM) CEO Mike Roman on Q1 2020 Res...,/article/4340658-3m-company-mmm-ceo-mike-roman...,2020-04-28,MMM,Industrials,Industrial Conglomerates,2020-05-26,2020-05-05,2020-04-29,157.610001,...,147.429993,156.259995,-0.035087,-0.064590,-0.008565,sell,sell,hold,company nyse earn conference call april compan...,hold
3,3M's (MMM) CEO Mike Roman on Q4 2019 Results -...,/article/4319588-3ms-mmm-ceo-mike-roman-on-q4-...,2020-01-28,MMM,Industrials,Industrial Conglomerates,2020-02-25,2020-02-04,2020-01-29,165.580002,...,157.240005,162.000000,-0.113117,-0.050368,-0.021621,sell,sell,hold,company nyse earn conference call january comp...,hold
4,3M Company (MMM) CEO Mike Roman on Q3 2019 Res...,/article/4298957-3m-company-mmm-ceo-mike-roman...,2019-10-24,MMM,Industrials,Industrial Conglomerates,2019-11-21,2019-10-31,2019-10-25,161.889999,...,164.990005,166.089996,0.020075,0.019149,0.025944,hold,hold,hold,company nyse earn conference call october comp...,hold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5268,"Zoetis, Inc. (ZTS) CEO Juan Ramon Alaix on Q1 ...",/article/4259182-zoetis-inc-zts-ceo-juan-ramon...,2019-05-02,ZTS,Health Care,Pharmaceuticals,2019-05-30,2019-05-09,2019-05-03,103.150002,...,101.669998,103.750000,-0.007950,-0.014348,0.005817,hold,hold,hold,zoetis inc. nyse earn conference call company ...,hold
5269,"Zoetis, Inc. (ZTS) CEO Juan Ramón Alaix on Q4 ...",/article/4241305-zoetis-inc-zts-ceo-juan-ramon...,2019-02-14,ZTS,Health Care,Pharmaceuticals,2019-03-14,2019-02-21,2019-02-15,93.260002,...,93.470001,95.370003,0.031525,0.002252,0.022625,buy,hold,hold,zoetis inc. nyse earn conference call february...,hold
5270,Zoetis (ZTS) Q3 2018 Results - Earnings Call T...,/article/4217256-zoetis-zts-q3-2018-results-ea...,2018-11-01,ZTS,Health Care,Pharmaceuticals,2018-11-29,2018-11-08,2018-11-02,94.250000,...,94.900002,92.769997,-0.015703,0.006897,-0.015703,hold,hold,hold,zoetis inc. nyse earn call november executive ...,hold
5271,Zoetis (ZTS) Q2 2018 Results - Earnings Call T...,/article/4193916-zoetis-zts-q2-2018-results-ea...,2018-08-02,ZTS,Health Care,Pharmaceuticals,2018-08-30,2018-08-09,2018-08-03,91.430000,...,92.400002,91.629997,-0.017719,0.010609,0.002187,hold,hold,hold,zoetis inc. nyse earn call august executive st...,hold


In [121]:
df_ECT_500_predict.to_csv('df_ECT_500_predict.csv')